# Dataset and Dataloader class in PyTorch

In [2]:
from sklearn.datasets import make_classification
import torch

In [3]:
# Step 1: Create a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)

In [4]:
X.shape

(10, 2)

In [5]:
# Convert the data to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [6]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        
        ## any transformations can be applied in this method
        ## for image transformation like resize, augmentation
        # for text data lemmatization
        return self.features[index], self.labels[index]        

In [8]:
dataset = CustomDataset(X,y)

In [9]:
len(dataset)

10

In [12]:
dataset[8]

(tensor([1.8997, 0.8344]), tensor(1))

In [ ]:
dataloader = DataLoader(dataset, batch_size=2, shuffle = True)

## dataloader is an iterable

In [16]:
for batch_features, batch_labels in dataloader:
    print(batch_features)
    print(batch_labels)
    print("=="*20)

tensor([[-0.9382, -0.5430],
        [ 1.7273, -1.1858]])
tensor([1, 1])
tensor([[ 1.0683, -0.9701],
        [ 1.7774,  1.5116]])
tensor([1, 1])
tensor([[-0.5872, -1.9717],
        [ 1.8997,  0.8344]])
tensor([0, 1])
tensor([[-1.1402, -0.8388],
        [-2.8954,  1.9769]])
tensor([0, 0])
tensor([[-0.7206, -0.9606],
        [-1.9629, -0.9923]])
tensor([0, 0])


### Using the brest cancer dataset

In [18]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [22]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [23]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [25]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [27]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [28]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

    def __init__(self, features, labels):

        self.features = features
        self.labels = labels

    def __len__(self):

        return len(self.features)

    def __getitem__(self, idx):

        return self.features[idx], self.labels[idx]



In [29]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [30]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Defining the model

In [31]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [32]:
learning_rate = 0.1
epochs = 25

In [33]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

### Training pipeline

In [35]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.20940443873405457
Epoch: 2, Loss: 0.13863199949264526
Epoch: 3, Loss: 0.0783277377486229
Epoch: 4, Loss: 0.39127033948898315
Epoch: 5, Loss: 0.07245396822690964
Epoch: 6, Loss: 0.14517782628536224
Epoch: 7, Loss: 0.0721948891878128
Epoch: 8, Loss: 0.182747021317482
Epoch: 9, Loss: 0.09636204689741135
Epoch: 10, Loss: 0.09556809812784195
Epoch: 11, Loss: 0.06431856751441956
Epoch: 12, Loss: 0.027326786890625954
Epoch: 13, Loss: 0.10851094871759415
Epoch: 14, Loss: 0.1826295703649521
Epoch: 15, Loss: 0.07694492489099503
Epoch: 16, Loss: 0.13357408344745636
Epoch: 17, Loss: 0.5747700333595276
Epoch: 18, Loss: 0.14323796331882477
Epoch: 19, Loss: 0.5953516364097595
Epoch: 20, Loss: 0.015060381963849068
Epoch: 21, Loss: 0.007361643947660923
Epoch: 22, Loss: 0.037090085446834564
Epoch: 23, Loss: 0.019980883225798607
Epoch: 24, Loss: 0.06531546264886856
Epoch: 25, Loss: 0.07628321647644043


In [36]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9549
